In [10]:
import torch
import torch.nn as nn
from collections import namedtuple

import sys
sys.path.append('..')
from utilities import BreakUp, Combine, combine_packed_sequence, struct_flatten, struct_unflatten, struct_equal
from block_rnn import SequenceStruct, BlockRNN
from torch.nn.utils.rnn import PackedSequence, pack_sequence, pad_packed_sequence



In [11]:
Globals = namedtuple("globals", "n_batch n_x n_in n_hidden n_out n_seq dtype var")
globals =  Globals(
        n_batch = 8,
        n_x = 2,
        n_in = 18,
        n_hidden = 32,
        n_out = 5,
        n_seq = 100,
        dtype = torch.double,
        var = 'PACKED',
    )


In [12]:
# output model
class Outp(nn.Sequential):

    def __init__(sel, n_hidden, n_out):
        super().__init__( nn.Linear(n_hidden ,10) ,nn.ELU() ,nn.Linear(10 ,n_out))

    def _forward(self, z, y):
        logits = super().forward(z)
        return -torch.distributions.Categorical(logits=logits).log_prob(y)

    def forward(self, z, y):
        if isinstance(z, torch.Tensor):
            return self._forward(z,y)
        else:
            l = PackedSequence(data=self._forward(z.data, y.data), batch_sizes=z.batch_sizes, 
                               sorted_indices=z.sorted_indices, unsorted_indices=z.unsorted_indices)    
            return l

outp = Outp(globals.n_hidden, globals.n_out).to(globals.dtype)

In [13]:
# rnn models
class FancyRNN(nn.Module):

    def __init__(self, n_in, n_hidden):
        super().__init__()
        self.gru = nn.GRU(n_in, 2* n_hidden, batch_first=True)
        self.lstm = nn.LSTM(2 * n_hidden, n_hidden, batch_first=True, num_layers=3)

    def forward(self, x, h):
        tmp, h_gru = self.gru(x, None if h is None else h[0])
        z, h_lstm = self.lstm(tmp, None if h is None else h[1])

        return z, (h_gru, h_lstm)



rnn = nn.GRU(globals.n_in, globals.n_hidden, batch_first=True).to(globals.dtype)
#rnn = nn.LSTM(globals.n_in, globals.n_hidden, batch_first=True, num_layers=3).to(globals.dtype)
#rnn = FancyRNN(globals.n_in, globals.n_hidden).to(globals.dtype)


In [19]:
def create_sequences(globals):

    if globals.var=='FIXED':
        lengths = torch.randint(globals.n_seq//5, globals.n_seq+1, (globals.n_batch,)) 
        x = pack_sequence([torch.randn(l, globals.n_in, dtype=globals.dtype) for l in lengths], enforce_sorted=False)
        y = pack_sequence([torch.randint(0, globals.n_out, (l,)) for l in lengths], enforce_sorted=False)
       
    else:
        x = torch.randn(globals.n_batch, globals.n_seq, globals.n_in, dtype=globals.dtype)
        y = torch.randint(0, globals.n_out, (globals.n_batch, globals.n_seq))

    return x,y    

x,y = create_sequences(globals)

In [21]:
        loss_scale = 'MEAN'
        
        mods = nn.ModuleList([outp, rnn])
        z, h = rnn(x, None)

        l_std = outp(z, y)
        if globals.var=='FIXED':
            l_std, lens = pad_packed_sequence(l_std, batch_first=True) 
            l_std = l_std.sum(1)
            if loss_scale == "MEAN":
                l_std /= lens
        else:
            l_std = l_std.mean(1) if loss_scale == "MEAN" else l_std.sum(1)
        print(l_std)
        l_std = l_std.sum()

        l_std.backward()
        l_std = l_std.item()
        g_std = [p.grad.clone() for p in mods.parameters()]
        print("loss (standard computation) {:.6f}".format(l_std))



tensor([1.6339, 1.6191, 1.6190, 1.6700, 1.6441, 1.6817, 1.5788, 1.6251],
       dtype=torch.float64, grad_fn=<MeanBackward1>)
loss (standard computation) 13.071750


In [22]:
        N=7
        
        mods.zero_grad()

        vlrnn = BlockRNN(rnn, outp, loss_scale)
        l_chunk = vlrnn(x, None, y,  N)
        g_chunk = [p.grad.clone() for p in mods.parameters()]
        print("loss (chunked computation) {:.6f}".format(l_chunk))


TypeError: 'int' object is not subscriptable

In [17]:
all([torch.allclose(g1, g2) for g1, g2 in zip(g_chunk, g_std )])


True

In [18]:
lens

tensor([21, 68, 63, 75, 25, 48, 26, 79])